In [2]:
from stockfish import Stockfish
import pandas as pd
import numpy as np

stockfish_path = "C:/Users/Matthew Saad/Documents/Stockfish/stockfish_15.1_win_x64_avx2/stockfish-windows-2022-x86-64-avx2.exe"

stockfish = Stockfish(path = stockfish_path, depth=6, parameters={"Threads": 8})

# Get fens (+ evals to compare)
df = pd.read_csv("../../data/kaggleDataset/chessData.csv", nrows = 1_000_000)

In [2]:
def get_eval_value(fen):
    stockfish.set_fen_position(fen)
    eval = stockfish.get_evaluation()

    if eval["type"] == "mate":
        if eval["value"] == 1:
            value = 3_000
        elif 2 <= eval["value"] < 10:
            value = 2_900 - (eval["value"] - 2) * 100
        else:
            value = 2_000
    else:
        value = max(min(eval["value"], 1900), -1900)
    
    return np.int16(value)

In [ ]:
def preprocess_scores(eval):
    score_dict = {
        '#+': 2000,
        '#-': -2000
    }

    try:
        eval = int(eval)
        eval = max(min(eval, 1500), -1500)
    except ValueError:
        eval = score_dict.get(eval[0:2], 0)

    return eval


In [3]:
evals = df.FEN.apply(get_eval_value)

np.save("data/createdData/npyFiles/stockFishEvalsChunk0", np.asarray(evals))

In [7]:
data = np.load("../../data/createdData/npyFiles/stockFishEvalsChunk0.npy")

In [11]:
len(data[data > 2_000])

1146